Fetch Website Content

In [3]:
!pip install requests --upgrade

In [4]:
import requests
import time
import random

In [5]:
web_url = "https://libraries.io"

In [6]:
response = requests.get(web_url)

In [7]:
response.status_code

200

In [8]:
len(response.text)

21765

In [9]:
page_contents = response.text

Parse Data

In [10]:
!pip install beautifulsoup4 --upgrade --quiet

In [11]:
from bs4 import BeautifulSoup

In [12]:
doc = BeautifulSoup(page_contents, 'html.parser')

Method1: Package Manager Popular Library Page

In [13]:
required_pkgs = ['wxpython', 'tensorflow', 'tfds', 'tensorboard', 'pytorch', 'numpy',
    'scipy', 'scikit-learn', 'pandas', 'keras', 'flask', 'tensorflow-transform', 'zappa',
    'subprocess', 'sqlalchemy', 'psycopg', 'flask-sqlalchemy', 'airflow', 'fftw', 'tensorflow-core-platform',
                 'cuda', 'cudnn', 'python', 'ubuntu', 'windows', 'macos', 'centos', 'android', 'ios', 'debian',
                 'bazel', 'glibc', 'jetpack', 'gcc', 'gzip', 'coremltools', 'postgres', 'mysql', 'blender', 'emr']

In [14]:
len(required_pkgs)

40

Method2: Search for Certain Packages

In [15]:
def get_response(url):
  response = requests.get(url)
  if (response.status_code != 200):
    print("Fetch Error, retrying ...")
    time.sleep(random.uniform(0, 5))
    response = requests.get(url)
    if (response.status_code != 200):
      print("FETCH ERROR AGAIN")
      return -1
  else:
    return response

Trial Example for Tensorflow

In [16]:
search_url = "https://libraries.io/search?q=" + "tensorflow"
search_response = get_response(search_url)
search_doc = BeautifulSoup(search_response.text, 'html.parser')
topResult_url = search_doc.find('div', {'class': 'project'}).find('h5').find('a')['href']
lib_url = web_url + topResult_url
print(lib_url)

https://libraries.io/conda/tensorflow


In [17]:
lib_search_response = get_response(lib_url)
lib_search_doc = BeautifulSoup(lib_search_response.text, 'html.parser')
detail_card = lib_search_doc.find('dl', {'class': 'row detail-card'})
key_list = detail_card.find_all('dt', {'class': 'col-xs-8'})
value_list = detail_card.find_all('dd', {'class': 'col-xs-4'})
href_dic = {}
print(len(key_list), len(value_list))

for i in range(len(key_list)):
    a_element = value_list[i].find('a')
    if (a_element):
        href_dic[key_list[i].text.strip()] = a_element['href']
print(href_dic)


12 12
{'Total releases': '/conda/tensorflow/versions', 'Stars': 'https://github.com/tensorflow/tensorflow/stargazers', 'Forks': 'https://github.com/tensorflow/tensorflow/network', 'Watchers': 'https://github.com/tensorflow/tensorflow/watchers', 'Contributors': 'https://github.com/tensorflow/tensorflow/graphs/contributors', 'SourceRank': '/conda/tensorflow/sourcerank'}


In [18]:
stats = obtain_stats(lib_url)
stats

NameError: name 'obtain_stats' is not defined

In [20]:
import json
stats_data = {}
for key, value in zip(key_list, value_list):
    key_text = key.text.strip()
    value_text = value.text.strip()

    if key_text in href_dic:
        stats_data[key_text] = {
            'value': value_text,
            'href': href_dic[key_text]
        }
    else:
        stats_data[key_text] = {
            'value': value_text
        }
stats_data

{'Dependencies': {'value': '5'},
 'Dependent packages': {'value': '59'},
 'Dependent repositories': {'value': '0'},
 'Total releases': {'value': '44', 'href': '/conda/tensorflow/versions'},
 'Latest release': {'value': 'May  2, 2023'},
 'First release': {'value': 'Jan 19, 2018'},
 'Stars': {'value': '177K',
  'href': 'https://github.com/tensorflow/tensorflow/stargazers'},
 'Forks': {'value': '88.9K',
  'href': 'https://github.com/tensorflow/tensorflow/network'},
 'Watchers': {'value': '7,693',
  'href': 'https://github.com/tensorflow/tensorflow/watchers'},
 'Contributors': {'value': '888',
  'href': 'https://github.com/tensorflow/tensorflow/graphs/contributors'},
 'Repository size': {'value': '887 MB'},
 'SourceRank': {'value': '19', 'href': '/conda/tensorflow/sourcerank'}}

In [21]:
main_column = lib_search_doc.find('div', {'class': 'col-md-8'}).find('dl')
keywords_list = main_column.find('dd').find_all('a')
license_list = main_column.find_all('dd')[1].find_all('a')
install_command = main_column.find('dd', {"class": "well"}).find('code').text.strip()

keywords = []
for keyword in keywords_list:
    keywords.append(keyword.text.strip())
license = []
for lic in license_list:
    license.append(lic.text.strip())
install_command


'conda install -c anaconda tensorflow'

In [22]:
keyword_title = main_column.find('dt').text
license_title = main_column.find_all('dt')[1].text
license_title

'\n      License\n    '

In [23]:
stats_data['keywords'] = keywords
stats_data['license'] = license
stats_data['install_command'] = install_command
stats_data

{'Dependencies': {'value': '5'},
 'Dependent packages': {'value': '59'},
 'Dependent repositories': {'value': '0'},
 'Total releases': {'value': '44', 'href': '/conda/tensorflow/versions'},
 'Latest release': {'value': 'May  2, 2023'},
 'First release': {'value': 'Jan 19, 2018'},
 'Stars': {'value': '177K',
  'href': 'https://github.com/tensorflow/tensorflow/stargazers'},
 'Forks': {'value': '88.9K',
  'href': 'https://github.com/tensorflow/tensorflow/network'},
 'Watchers': {'value': '7,693',
  'href': 'https://github.com/tensorflow/tensorflow/watchers'},
 'Contributors': {'value': '888',
  'href': 'https://github.com/tensorflow/tensorflow/graphs/contributors'},
 'Repository size': {'value': '887 MB'},
 'SourceRank': {'value': '19', 'href': '/conda/tensorflow/sourcerank'},
 'keywords': ['deep-learning',
  'deep-neural-networks',
  'distributed',
  'machine-learning',
  'ml',
  'neural-network',
  'python',
  'tensorflow'],
 'license': ['Apache-2.0'],
 'install_command': 'conda install

Functions for all packages

In [67]:
revised_required_pkgs = ['wxpython', 'tensorflow', 'tensorflow-datasets', 'tensorboard', 'pytorch', 'numpy',
    'scipy', 'scikit-learn', 'pandas', 'keras', 'flask', 'zappa',
    'subprocess', 'sqlalchemy', 'psycopg', 'flask-sqlalchemy', 'tensorflow-core-platform']
outlier_required_pkgs = ['tensorflow-transform', 'airflow', 'fftw']

In [68]:
def obtain_lib_url(pkg):
    search_url = "https://libraries.io/search?q=" + pkg
    search_response = get_response(search_url)
    search_doc = BeautifulSoup(search_response.text, 'html.parser')
    topResult_url = search_doc.find('div', {'class': 'project'}).find('h5').find('a')['href']
    lib_url = web_url + topResult_url
    return lib_url

In [98]:
def obtain_ourlier_lib_url(pkg):
    search_url = "https://libraries.io/search?q=" + pkg
    search_response = get_response(search_url)
    search_doc = BeautifulSoup(search_response.text, 'html.parser')
    topResult_url = search_doc.find_all('div', {'class': 'project'})[1].find('h5').find('a')['href']
    lib_url = web_url + topResult_url
    return lib_url

In [83]:
def check_keyword_index(main_titles):
    for i in range(len(main_titles)):
        if (main_titles[i].text.strip() == "Keywords"):
            return i
    return -1

In [84]:
def check_license_index(main_titles):
    for i in range(len(main_titles)):
        if (main_titles[i].text.strip() == "License"):
            return i
    return -1

In [99]:
def obtain_stats(lib_url):
    lib_search_response = get_response(lib_url)
    lib_search_doc = BeautifulSoup(lib_search_response.text, 'html.parser')
    detail_card = lib_search_doc.find('dl', {'class': 'row detail-card'})
    key_list = detail_card.find_all('dt', {'class': 'col-xs-8'})
    value_list = detail_card.find_all('dd', {'class': 'col-xs-4'})
    href_dic = {}

    for i in range(len(key_list)):
        a_element = value_list[i].find('a')
        if (a_element):
            href_dic[key_list[i].text.strip()] = a_element['href']
    stats_data = {}
    for key, value in zip(key_list, value_list):
        key_text = key.text.strip()
        value_text = value.text.strip()

        if key_text in href_dic:
            stats_data[key_text] = {
                'value': value_text,
                'href': href_dic[key_text]
            }
        else:
            stats_data[key_text] = {
                'value': value_text
            }
    
    main_column = lib_search_doc.find('div', {'class': 'col-md-8'}).find('dl')
    main_titles = main_column.find_all('dt')
    keyword_index= check_keyword_index(main_titles)
    license_index = check_license_index(main_titles)

    if (keyword_index != -1):
        keyword_list = main_column.find_all('dd')[keyword_index].find_all('a')
        keywords = []
        for keyword in keyword_list:
            keywords.append(keyword.text.strip())
        stats_data['keywords'] = keywords
    else:
        stats_data['keywords'] = []
    
    if (license_index != -1):
        license_list = main_column.find_all('dd')[license_index].find_all('a')
        license = []
        for lic in license_list:
            license.append(lic.text.strip())
        stats_data['license'] = license
    else:
        stats_data['license'] = []

    try:
        install_command = main_column.find('dd', {"class": "well"}).find('code').text.strip()
    except AttributeError:
        install_command = ""
    stats_data['install_command'] = install_command
    
    return stats_data
    

In [101]:
look_up_dic = {}

In [102]:
fetch_count = 0
for pkg in revised_required_pkgs:
  print(pkg)
  fetch_count += 1
  if fetch_count % 5 == 0:
    time.sleep(random.uniform(0, 200))
  lib_url = obtain_lib_url(pkg)
  look_up_dic[pkg] = obtain_stats(lib_url)

for pkg in outlier_required_pkgs:
  fetch_count += 1
  if fetch_count % 5 == 0:
    time.sleep(random.uniform(0, 200))
  lib_url = obtain_ourlier_lib_url(pkg)
  look_up_dic[pkg] = obtain_stats(lib_url)
look_up_dic["subprocess"]

wxpython
tensorflow
tensorflow-datasets
tensorboard
pytorch
numpy
scipy
scikit-learn
pandas
keras
flask
zappa
subprocess
sqlalchemy
psycopg
flask-sqlalchemy
tensorflow-core-platform


{'Dependencies': {'value': '0'},
 'Dependent packages': {'value': '156'},
 'Dependent repositories': {'value': '67'},
 'Total releases': {'value': '31', 'href': '/cargo/subprocess/versions'},
 'Latest release': {'value': 'May 15, 2022'},
 'First release': {'value': 'Jan 14, 2017'},
 'Stars': {'value': '381',
  'href': 'https://github.com/hniksic/rust-subprocess/stargazers'},
 'Forks': {'value': '41',
  'href': 'https://github.com/hniksic/rust-subprocess/network'},
 'Watchers': {'value': '6',
  'href': 'https://github.com/hniksic/rust-subprocess/watchers'},
 'Contributors': {'value': '15',
  'href': 'https://github.com/hniksic/rust-subprocess/graphs/contributors'},
 'Repository size': {'value': '383 KB'},
 'SourceRank': {'value': '17', 'href': '/cargo/subprocess/sourcerank'},
 'keywords': ['command', 'process', 'pipe', 'execute', 'redirect'],
 'license': [],
 'install_command': ''}

In [103]:
look_up_dic["subprocess"]

{'Dependencies': {'value': '0'},
 'Dependent packages': {'value': '156'},
 'Dependent repositories': {'value': '67'},
 'Total releases': {'value': '31', 'href': '/cargo/subprocess/versions'},
 'Latest release': {'value': 'May 15, 2022'},
 'First release': {'value': 'Jan 14, 2017'},
 'Stars': {'value': '381',
  'href': 'https://github.com/hniksic/rust-subprocess/stargazers'},
 'Forks': {'value': '41',
  'href': 'https://github.com/hniksic/rust-subprocess/network'},
 'Watchers': {'value': '6',
  'href': 'https://github.com/hniksic/rust-subprocess/watchers'},
 'Contributors': {'value': '15',
  'href': 'https://github.com/hniksic/rust-subprocess/graphs/contributors'},
 'Repository size': {'value': '383 KB'},
 'SourceRank': {'value': '17', 'href': '/cargo/subprocess/sourcerank'},
 'keywords': ['command', 'process', 'pipe', 'execute', 'redirect'],
 'license': [],
 'install_command': ''}

Convert Dic to Json File

In [104]:
import json

In [105]:
STATS = {}

In [93]:
for i in look_up_dic.keys():
    print(i, end="\t")

wxpython	tensorflow	tensorflow-datasets	tensorboard	pytorch	numpy	scipy	scikit-learn	pandas	keras	flask	zappa	subprocess	sqlalchemy	psycopg	flask-sqlalchemy	tensorflow-core-platform	tensorflow-transform	airflow	fftw	

In [94]:
test_list = ['a','b', 'c']
str = " "
print(str.join(test_list))

a b c


In [106]:
# for key, value in look_up_dic.items():

#     for item_key, item_value in value.items():

#         if item_key == 'keywords':
#             print(item_value)
#             if len(item_value) == []:
#                 value[item_key] = ""
#             else:
#                 keywords_string = ""
#                 for i in range(len(item_value)):
#                     keywords_string += item_value[i] + " "
#                 value[item_key] = keywords_string

#         if item_key == 'license':
#             value[key] = item_value[0]
        
#         if item_key != 'keywords' and item_key != 'license':
#             value[key] = item_value
#     STATS[key] = value

for pri_key, pri_value in look_up_dic.items():
    print(pri_key)
    if len(pri_value['keywords']) == 0:
        pri_value['keywords'] = ""
    else:
        pri_value['keywords'] = " ".join(pri_value['keywords'])
    if len(pri_value['license']) == 0:
        pri_value['license'] = ""
    else:
        pri_value['license'] = pri_value['license'][0]


    STATS[pri_key] = pri_value

wxpython
tensorflow
tensorflow-datasets
tensorboard
pytorch
numpy
scipy
scikit-learn
pandas
keras
flask
zappa
subprocess
sqlalchemy
psycopg
flask-sqlalchemy
tensorflow-core-platform
tensorflow-transform
airflow
fftw


In [96]:
STATS[key]

{'Dependencies': {'value': '0'},
 'Dependent packages': {'value': '52.5K'},
 'Dependent repositories': {'value': '84.9K'},
 'Total releases': {'value': '152', 'href': '/pypi/numpy/versions'},
 'Latest release': {'value': '23 days ago'},
 'First release': {'value': 'Dec  2, 2006'},
 'Stars': {'value': '24.2K',
  'href': 'https://github.com/numpy/numpy/stargazers'},
 'Forks': {'value': '8.39K', 'href': 'https://github.com/numpy/numpy/network'},
 'Watchers': {'value': '592',
  'href': 'https://github.com/numpy/numpy/watchers'},
 'Contributors': {'value': '905',
  'href': 'https://github.com/numpy/numpy/graphs/contributors'},
 'Repository size': {'value': '122 MB'},
 'SourceRank': {'value': '31', 'href': '/pypi/numpy/sourcerank'},
 'keywords': 'numpy python',
 'license': 'Other',
 'install_command': 'pip install numpy==1.26.0'}

In [107]:
file_path = 'stats.json'
with open(file_path, 'w') as json_file:
    json.dump(STATS, json_file, indent=4)